In [1]:
import os
os.chdir('..')
from jack import readers
from projects.assertion_mr.shared import AssertionMRPorts
from projects.assertion_mr.qa.definition_model import DefinitionPorts
from jack.io.load import load_squad
from jack.util import preprocessing

/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import tensorflow as tf
tf.reset_default_graph()
reader = readers.reader_from_file('/home/diwe01/fusessh/r24nlu/triviaqa_wiki/models/3/bilstm_conceptnet_wiki/')


INFO:tensorflow:Restoring parameters from /home/diwe01/fusessh/r24nlu/triviaqa_wiki/models/3/bilstm_conceptnet_wiki/model_module


In [3]:
from jack.io.load import load_jack

dataset = load_jack('../jtr/data/triviaqa/wiki-dev.json')

In [4]:

batch_size = 4

for index in range(0, 1028, batch_size):
    question_batch = [q for q, a in dataset[index:index + batch_size]]
    answer_batch = [a for q, a in dataset[index:index + batch_size]]
    
    predicted_answers_wiki = reader(question_batch)
    reader.input_module.use_definitions = False
    predicted_answers = reader(question_batch)
    reader.input_module.use_definitions = True
    for paw, pa, ass, q in zip(predicted_answers_wiki, predicted_answers, answer_batch, question_batch):
        if any(pa[0].text == a.text for a in ass) and all(paw[0].text != a.text for a in ass):
            print("============================= Wrong after Definition ================================")
            print('Question:               ', q.question)
            print('Top answers candidates: ')
            for a in set(a.text for a in pa):
                print('    -', a)
            print()
            print('Answer before Wikipedia:', pa[0].text)
            print('Answer after Wikipedia: ', paw[0].text)
            print()
            print('== Read Wikipedia Definitions ==')
            b = reader.input_module([q])
            rev_vocab = b['__rev_vocab']
            for d, l in zip(b[DefinitionPorts.definitions], b[DefinitionPorts.definition_lengths]):
                print('-', ' '.join(rev_vocab[i] for i in d[:l]))         
            print('=====================================================================================')
            print()
        elif any(paw[0].text == a.text for a in ass) and all(pa[0].text != a.text for a in ass):
            print("=========================== Corrected after Definition ==============================")
            print('Question:               ', q.question)
            print('Top answers candidates: ')
            for a in set(a.text for a in pa):
                print('    -', a)
            print()
            print('Answer before Wikipedia:', pa[0].text)
            print('Answer after Wikipedia: ', paw[0].text)
            print()
            print('== Read Wikipedia Definitions ==')
            b = reader.input_module([q])
            rev_vocab = b['__rev_vocab']
            for d, l in zip(b[DefinitionPorts.definitions], b[DefinitionPorts.definition_lengths]):
                print('-', ' '.join(rev_vocab[i] for i in d[:l]))           
            print('=====================================================================================')
            print()

=========================== Corrected after Definition ==============================
Question:                What claimed the life of singer Kathleen Ferrier ?
Top answers candidates: 
    - contralto
    - pianist , and won numerous amateur piano
    - piano competitions while working as a telephonist
    - artist
    - international reputation
    - cancer
    - piano
    - telephonist
    - pianist

Answer before Wikipedia: telephonist
Answer after Wikipedia:  cancer

== Read Wikipedia Definitions ==
- Cancer is an album by My Disco , released on November 7 , 2006
- \"Cancer\ " is a song by American rock band My Chemical Romance
- Cancer are a British death / thrash metal band formed in Ironbridge , Telford , Shropshire in 1988
- Piano is a 1990 play by Trevor Griffiths , adapted from the 1977 film An Unfinished Piece for Mechanical Piano , itself based on the incomplete and untitled early Anton Chekhov play usually known as Platonov
- The piano is an acoustic , stringed musical i

============================= Wrong after Definition ================================
Question:                Where did the Shinning Path terrorists operate ?
Top answers candidates: 
    - Peru
    - Canada

Answer before Wikipedia: Peru
Answer after Wikipedia:  Canada

== Read Wikipedia Definitions ==
- Peru ( also known as Peru the Band ) is an indie rock trio formed in Long Island , New York in 2014
- Peru ( /pəˈruː/ ; Spanish : Perú [ peˈɾu ] ; Quechua : Piruw [ pɪɾʊw ] ; Aymara : Piruw [ pɪɾʊw ] ) , officially the Republic of Peru ( Spanish : ) , is a country in western South America
- Canada is an American indie folk - pop music group from Ann Arbor , Michigan
- Rabbi Ephraim Buchwald is one of the leaders in the movement of orthodox Jewish outreach in America today .
- The cinema of Canada or Canadian cinema refers to the filmmaking industry in Canada

=========================== Corrected after Definition ==============================
Question:                Who had a 70s N

============================= Wrong after Definition ================================
Question:                Beloved in 1999 was whose first movie since The Color Purple in 1985 ?
Top answers candidates: 
    - Oprah Winfrey
    - Steven Spielberg
    - Alice Walker
    - Whoopi Goldberg
    - Thandie Newton

Answer before Wikipedia: Oprah Winfrey
Answer after Wikipedia:  Thandie Newton

== Read Wikipedia Definitions ==
- Oprah Gail Winfrey ( born Orpah Gail Winfrey ; January 29 , 1954 ) is an American media proprietor , talk show host , actress , producer , and philanthropist
- Melanie Thandiwe \"Thandie\ " Newton ( born 6 November 1972 ) is an English actress
- Steven Allan Spielberg KBE OMRI ( born December 18 , 1946 ) is an American director , producer , screenwriter , and editor
- Alice Walker was a British fencer
- Alice Malsenior Walker ( born February 9 , 1944 ) is an American novelist , short story writer , poet , and activist
- Caryn Elaine Johnson ( born November 13 , 1955

=========================== Corrected after Definition ==============================
Question:                Who was the star of the film based on the record Harper Valley PTA by Jeannie C Riley ?
Top answers candidates: 
    - Tom T. Hall
    - Dolly Parton
    - Barbara Eden
    - Mrs. Johnson
    - Barbara Eden as the widow Mrs. Johnson

Answer before Wikipedia: Barbara Eden as the widow Mrs. Johnson
Answer after Wikipedia:  Barbara Eden

== Read Wikipedia Definitions ==
- Barbara Eden ( born August 23 , 1931 ) is an American film , stage , and television actress , and singer , best - known for her starring role of \"Jeannie\ " in the sitcom I Dream of Jeannie .
- Johnson ( 26 March 1953 – 18 August 2011 ) was an Indian film score composer and music director who has given music to some of the most important motion pictures of Malayalam cinema , including those for Koodevide , Namukku Parkkan Munthiri Thoppukal , Oru Minnaminunginte Nurunguvettam , Vadakkunokkiyantram , Perumthacha

KeyboardInterrupt: 

In [1]:
import os
os.chdir('..')
from projects.assertion_mr.assertions.store import AssertionStore

In [2]:
s = AssertionStore('/data/assertion_store')

In [7]:
for e in s.assertion_keys_for_subject('London', 'wikipedia_firstsent'):
    print(s.get_assertion(e))

In [20]:
for r, s2a in s._object2assertions.items():
    for b, a in s2a.items():
        s2a[b] = set(a)

In [23]:
s.save()